# Retreiving and Preparing Stock's Income Statement Information
This section of the code will concentrate on extracting stock's income statement information from IEX Cloud API. At the end, this section of the code provide the complete and formatted income statement dataset.  

## Importing neccessary modules

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import xlsxwriter

## Setting up base URL

Fetching the fundamentals and finacial data about the stock from ther IEX Cloud endpoints.

In [3]:
from api_secret import IEX_CLOUD_API_TOKEN

symbol = 'FTNT'#input("Enther the ticker here: ")
income_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/income?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
financials_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/financials?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
fundamentals_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/fundamentals?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'

income_endpoint_data = requests.get(income_endpoint).json()
financials_endpoint_data = requests.get(financials_endpoint).json()
fundamentals_endpoint_data = requests.get(fundamentals_endpoint).json()

## Extracting the data
This part will concentrate on extracting the data that will form the final income statement for the stock.

### Forming the Dataset columns
The columns will be comprised of the Pandas DatetimeIndex. It will be created based on the quarters (reportDate) of the reported stock data.


First step is to find the quarters of the reported stock data.

In [4]:
reportDate_list = []
quarter_list = []

for stock_data in income_endpoint_data['income']:
    converted_date = datetime.strptime(stock_data['reportDate'], '%Y-%m-%d').date()
    reportDate_list.append(converted_date)

def quarter_finder(dates_list):
    global quarter_list

    for d in reportDate_list:
        q1 = date(d.year, 3, 31)
        q2 = date(d.year, 6, 30)
        q3 = date(d.year, 9, 30)
        q4 = date(d.year, 12, 31)

        if d < q1:
            q = date(d.year - relativedelta(year=1).year, 12, 31)
        if d > q1 and d < q2:
            q = date(d.year, 3, 30)
        if d > q2 and d < q3:
            q = date(d.year, 6, 30)
        if d > q3 and d < q4:
            q = date(d.year, 9, 30)
        
        quarter_list.append(q)

quarter_finder(reportDate_list)

The above created the Datetime-like array which will serve as an input to Pandas' DatetimeIndex objet (columns of the future dataset)

In [5]:
columns = pd.DatetimeIndex(data=quarter_list)

The next step is the creation of the actual DataFrame.

In [6]:
index = [
    'Total Revenue',
    'Total Revenue (Growth)',
    'Cost of Revenue',
    'Cost of Revenue (Growth)',
    'Gross Profit',
    'Gross Profit (Growth)',
    'Operating Expense',
    'SG&A',
    'SG&A (Growth)',
    'Research and Development',
    'Operating Income',
    'Pretax Income',
    'Pretax Income (Growth)',
    'Tax Provision',
    'Net Income',
    'Net Income (Growth)',
    'Basic EPS',
    'Basic EPS (Growth)',
    'Diluted EPS',
    'Total Expenses',
    'Total Expenses (Growth)',
    'Interest Income',
    'Inerest Income (Growth)',
    'Interest Expense',
    'Interest Expense (Growth)',
    'EBIT',
    'EBITDA',
    'EBITDA (Growth)'
]

income_statement = pd.DataFrame(columns=columns, index=index, data=None)

Filling the dataframe with data returned by API endpoints.

The first API endpoint the results of which the iteraion will be made is the financials API endpoint.

In [7]:
financials_dict = {
    'Total Revenue': [],
    'Operating Income': [],
    'Pretax Income': [],
    'Tax Provision': [],
    'Net Income': [],
    'Total Expenses': [],
    'Interest Income': [],
    'EBIT': [],
    'EBITDA': []
}

for quarter_data in financials_endpoint_data['financials']:
    financials_dict['Total Revenue'].append(quarter_data['totalRevenue'])
    financials_dict['Operating Income'].append(quarter_data['operatingIncome'])
    financials_dict['Pretax Income'].append(quarter_data['pretaxIncome'])
    financials_dict['Tax Provision'].append(quarter_data['incomeTax'])
    financials_dict['Net Income'].append(quarter_data['netIncome'])
    financials_dict['Total Expenses'].append(quarter_data['operatingExpense'])
    financials_dict['Interest Income'].append(quarter_data['interestIncome'])
    financials_dict['EBIT'].append(quarter_data['ebit'])
    financials_dict['EBITDA'].append(quarter_data['EBITDA'])

for stock_attribute in financials_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = financials_dict[stock_attribute]


The second is the income API endpoint.

In [8]:
income_dict = {
    'Cost of Revenue': [],
    'Gross Profit': [],
    'Operating Expense': [],
    'SG&A': [],
    'Research and Development': []
}

for quarter_data in income_endpoint_data['income']:
    income_dict['Cost of Revenue'].append(quarter_data['costOfRevenue'])
    income_dict['Gross Profit'].append(quarter_data['grossProfit'])
    income_dict['Operating Expense'].append(quarter_data['operatingExpense'])
    income_dict['SG&A'].append(quarter_data['sellingGeneralAndAdmin'])
    income_dict['Research and Development'].append(quarter_data['researchAndDevelopment'])

for stock_attribute in income_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = income_dict[stock_attribute]

The final endpoint to use the data from in the dataset is the fundamentals endpoint.

In [9]:
fundamentals_dict = {
    'Basic EPS': [],
    'Diluted EPS': [],
    'Interest Expense': []
}

for quarter_data in fundamentals_endpoint_data['fundamentals']:
    fundamentals_dict['Basic EPS'].append(quarter_data['incomeNetPerWabsoSplitAdjusted'])
    fundamentals_dict['Diluted EPS'].append(quarter_data['incomeNetPerWadsoSplitAdjusted'])
    fundamentals_dict['Interest Expense'].append(quarter_data['expensesInterest'])

for stock_attribute in fundamentals_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = fundamentals_dict[stock_attribute]

## Calculating the Growth

In [10]:
total_revenue_growth = []

for df_index, row_name in enumerate(income_statement.index):
    if 'Growth' in row_name:
        for row_index, value in enumerate(income_statement.iloc[df_index - 1]):
            try:
                growth_rate = (value / income_statement.iloc[df_index - 1, row_index + 1]) * 100
                income_statement.iloc[df_index, row_index] = growth_rate - 100
            except IndexError:
                break

income_statement

,2022-09-30,2022-06-30,2022-03-30,2021-12-31,2021-09-30,2021-06-30,2021-03-30,2020-12-31,2020-09-30,2020-06-30,2020-03-30,2019-12-31
Total Revenue,1158817041,1045161098,970039003,988346905,907372386,824653701,719387760,771172449,654622477,630286908,603380023,614904298
Total Revenue (Growth),10.87449,7.744234,-1.852376,8.924067,10.030718,14.632712,-6.71506,17.804151,3.86103,4.45936,-1.874157,NaN
Cost of Revenue,295953889,261377711,265388337,232568100,218458648,190349772,160252702,170523440,143811652,136928207,130318724,145727848
Cost of Revenue (Growth),13.228434,-1.511229,14.112097,6.458637,14.766961,18.781006,-6.023065,18.574147,5.027047,5.071783,-10.573905,NaN
Gross Profit,909449512,817689588,716422476,752687724,687273405,625998105,561014738,606198151,530312782,480959408,461160885,480919576
Gross Profit (Growth),11.221853,14.135111,-4.8181,9.517947,9.788416,11.583184,-7.453572,14.309549,10.261443,4.293192,-4.108523,NaN
Operating Expense,905236383,837038413,843906077,762386238,722871332,687195823,609616226,595783901,549335674,508986289,504894800,496205790
SG&A,485896010,467786751,439402018,412619758,397936634,376260963,337408984,339184577,308378062,283470546,296001499,285314795
SG&A (Growth),3.871264,6.459855,6.490785,3.689815,5.760808,11.514803,-0.523489,9.989853,8.786633,-4.233409,3.745584,NaN
Research and Development,138305655,127578096,128462002,116551624,110064956,107303511,97712146,91566335,91046324,82454035,81725426,74349666


## Putting it all together into the Excel

This part of the code will concentrate on formatting the above data and organizing it in the Excel document via xlsxwriter.

Creating a **writer** object and writing the dataset into the 'Income Statement' sheet.

In [22]:
writer = pd.ExcelWriter('fundamental_analysis.xlsx', engine='xlsxwriter')
income_statement.to_excel(writer, sheet_name='Income Statement', startrow=1, header=False)

### Formatting

The next part of the code will create formats for the cells of the Excel file. Here's the list of the formats that will be used:

- String format for rows' names
- Date format for columns
- Integer format for values
- Percentage format for growth

In [23]:
standard_font_color = '#000000'
background = '#FFFFFF'
columns_background = '#92CDDC'
indices_background = '#FABF8F'

indices_format = writer.book.add_format(
    {
        'font_color': standard_font_color,
        'bg_color': indices_background,
        'border': 1,
        'bold': True
    }
)

header_format = writer.book.add_format(
    {
        'bold': True,
        'font_color': standard_font_color,
        'bg_color': columns_background,
        'border': 1,
        'num_format': 'yyyy-mm-dd'
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': standard_font_color,
        'bg_color': background,
        'border': 1,
    }
)

percentage_format = writer.book.add_format(
    {
        'num_format': '0%',
        'font_color': standard_font_color,
        'bg_color': background,
        'border': 1,
    }
)

Modifying the headers as well as the columns width.

In [24]:
for header_index, header in enumerate(income_statement.columns):
    writer.sheets['Income Statement'].write(0, header_index + 1, header, header_format)
    writer.sheets['Income Statement'].set_column(header_index + 1, header_index + 1, 18)

Changing the rows formats.

In [25]:
# Column A's width adjustment
writer.sheets['Income Statement'].set_column(0, 0, 30)

for col_num, col_name in enumerate(income_statement.columns):
    for row_num, row_name in enumerate(income_statement.index):
        writer.sheets['Income Statement'].write(row_num + 1, 0, row_name, indices_format)
        if 'Growth' in row_name:
            writer.sheets['Income Statement'].write(row_num + 1, col_num + 1, col_name, percentage_format)
        else:
            writer.sheets['Income Statement'].write(row_num + 1, col_num + 1, col_name, integer_format)

writer.save()

C:\Users\vadim\AppData\Local\Temp\ipykernel_636\1572393908.py:12: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


The next section will address the formatting for the stock's data as well as the Growth rows.